<a href="https://colab.research.google.com/github/omotayoofere/Muisc_Information_Retrieval/blob/main/MusicGenreClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from keras.datasets import fashion_mnist
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.layers import Dense, Conv2D, Activation, Flatten, MaxPooling2D

from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
from keras.layers import Dense, Conv2D, Activation, Flatten, MaxPooling2D


import time

import os
import glob
import random
import cv2 as cv
import numpy as np
import pandas as pd
from PIL import Image
from numpy import asarray



#from tensorflow import keras
from sklearn.model_selection import train_test_split

from tensorflow import keras
from keras import layers, models
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from sklearn.model_selection import cross_val_score


#EarlyStopping
from keras.callbacks import EarlyStopping

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#gdrive_path --> Google drive path where the images are saved
def structure_dataset(gdrive_path):
  categories = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
  data = []
  label = []

  for x in categories:
      path = gdrive_path + f'/{x}' + '/*.png' 
      #used to check for extensions in folders
      for file in glob.glob(path):
        #reading the image and converting to greyscale
        img = cv.imread(file, cv.IMREAD_GRAYSCALE)

        #Resizing images
        IMG_SIZE = 350
        image = cv.resize(img, (IMG_SIZE, IMG_SIZE))

        #Appends the image to the container holding the newly sized images
        data.append(image)
        #Converts image to an array
        X = np.asarray(data)

        #Appends array for each image to a container
        label.append(x)

        #Giving a numeric label to categories of image dataset
        label_dict = {
            'blues': 0,
            'classical': 1,
            'country': 2,
            'disco': 3,
            'hiphop': 4,
            'jazz': 5,
            'metal': 6,
            'pop': 7,
            'reggae': 8,
            'rock': 9,
        }

        #mapping the image labels and the numeric labels created
        y = np.array(list(map(label_dict.get, label)))

  return X, y

In [ ]:
X,y = structure_dataset('/content/gdrive/MyDrive/input/CNN_data')

In [ ]:
X = np.array(X / 255.0)

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
#x_train = x_train.reshape(len(x_train), 360, 360, 1)
#x_test = x_test.reshape(len(x_test), 360, 360, 1)

In [ ]:
X = X.reshape(len(X), 350, 350, 1)


In [ ]:
#An approach to augument data by presenting different form of the data to the model
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        #rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        #horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X)

In [ ]:
def build_model():
  model = keras.models.Sequential()

  model.add(Conv2D(32, (3,3), input_shape=X.shape[1:]))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))

  model.add(Conv2D(32, (3,3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  #model.add((Dropout(0.1)))

  model.add(Flatten())

  model.add(Dense(200, activation="relu"))
  #model.add((Dropout(0.2)))

  model.add(Dense(150, activation="relu"))
  #model.add((Dropout(0.1)))

  model.add(Dense(100, activation="relu"))
  #model.add((Dropout(0.1)))

  model.add(Dense(50, activation="relu"))
  #model.add((Dropout(0.1)))


  model.add(Dense(10))
  model.add(Activation('softmax'))

  model.compile(optimizer="adam",
                loss="sparse_categorical_crossentropy",
                metrics=["accuracy"])
  
  return model

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
scores = []
actual = []
preds = []

def evaluate_model(X, y):

  kfold = StratifiedKFold(n_splits=10, random_state=random.seed(101), shuffle=True)

  current_fold = 0
  for train, test in kfold.split(X,y):
      current_fold += 1
      print('Training fold %d' % current_fold)
      
      model = build_model()
      
      train_X, train_y, test_X, test_y = X[train], y[train], X[test], y[test]

      #Extract a 20% slot from training set for validation
      tr, val = next(StratifiedKFold(n_splits=5, shuffle=True).split(train_X, train_y))
      tr_X, tr_y, val_X, val_y = train_X[tr], train_y[tr], train_X[val], train_y[val]

      Es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)

      history = model.fit(tr_X, tr_y, epochs=100, batch_size=50, validation_data=(val_X, val_y), verbose=0, callbacks=[Es])
      
      _, acc = model.evaluate(test_X, test_y, verbose=0)

      print('>> %.3f' % (acc * 100.0))

      scores.append(acc)
      preds.append(history)

  print("%.2f%% (+/- %.2f%%)" % (np.mean(scores), np.std(scores)))
  return scores, preds

In [ ]:
def summarize(histories):
  for i in range(len(histories)):
    plt.figure()
    plt.subplot(211)
    plt.title('Cross Entropy Loss')
    plt.plot(histories[i].history['loss'], color='blue', label='train')
    plt.plot(histories[i].history['val_loss'], color='red', label='val')

    plt.subplot(212)
    plt.title('Classification Accuracy')
    plt.plot(histories[i].history['accuracy'], color='blue', label='train')
    plt.plot(histories[i].history['val_loss'], color='red', label='val')
    plt.show()

In [ ]:
def summarize_performance(scores):
  print('Accuracy: mean=%.3f std=%.3f, n=%d' % (np.mean(scores)*100, np.std(scores)*100, len(scores)))
  plt.boxplot(scores)
  plt.show()

In [ ]:
def run():
  scores, histories = evaluate_model(X, y)
  summarize(histories)
  summarize_performance(scores)

In [ ]:
run()